# Code to scrap indiakannon.org

In [2]:
# # work on select random document. Select those documents. 
# import random

# Maxnum = 2000001
# random_numbers = list(random.sample(range(1, Maxnum), 11))
# random_numbers.sort()
# print(random_numbers)

In [3]:
# Import Libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import pyautogui
import os 
import time
import pandas as pd 
import re

In [4]:
def getDriver():
    # Set up Selenium WebDriver with the ChromeDriver executable path
    driver_path = r'C:/Users/hari_/Downloads/chromedriver_win32/chromedriver.exe'

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')  # Run Chrome in headless mode
    chrome_options.add_argument(f'--download.default_directory={download_path}')
    chrome_options.add_experimental_option('prefs', {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
    })

    # Initialize the driver with the configured options
    driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)
    return driver 

In [5]:

# # For debugging function
# driver = getDriver()
# doc_id =217104
# url = f'https://indiankanoon.org/doc/{doc_id}/'
# driver.get(url)

def get_citations(driver):
    cited_by = []
    cited_content=[] 
    no_of_citation=""
    try:
        try: # if page there are many citation and there is search page
            no_of_citation = driver.find_elements('css selector', 'div.doc_cite_head')[0].text.split("-")[0]
            cite_url = driver.find_elements('css selector', 'div.doc_cite_head')[0].find_element('xpath', './a').get_attribute('href')
        except:
            cite_url = ""

        if cite_url=="": # collection citation information from the same page
            try:
                single_cited_by = driver.find_elements('css selector', 'div.cite_title')
                cited_by.extend(single_cited_by)
                for cite in cited_by:
                    try:
                        #print(cite.text)
                        cited_content.append(cite.text + " : " + cite.find_element('xpath', './a').get_attribute('href'))
                    except:
                        pass
            except NoSuchElementException:
                pass
        else: # collect citzation informatio from other page, search page.
            driver1 = getDriver()
            driver1.get(cite_url)

            single_cited_by=driver1.find_elements('css selector', 'div.result')        
            cited_by.extend(single_cited_by)

            for cite in cited_by:
                try:
                    #print(cite.text)
                    cited_content.append(cite.text) #+ " : " + cite.find_element('xpath', './a').get_attribute('href') + "\n"
                except:
                    pass

    except NoSuchElementException:
        pass
    return no_of_citation, cited_content

In [6]:
# # Code below is for debugging 
# driver = getDriver()
# doc_id =217104
# url = f'https://indiankanoon.org/doc/{doc_id}/'
# driver.get(url)
# data = []
import os 

def savein_textfile(doc_id, url, driver, data):
    junk_txt = 'Take notes as you read a judgment using our Virtual Legal Assistant and get email alerts whenever a new judgment matches your query (Query Alert Service). Try out our Premium Member services -- Free for one month.'
    # Extract the content
    content_element=""
    doctype = ""

    # If judgement then get judgement details
    try:
        content_element = driver.find_element('css selector', 'div.judgments')
        doctype = "Judgement"
    except NoSuchElementException:
            pass     

    # if act then get act details
    if doctype == "":
        try:
            content_element = driver.find_element('css selector', 'div.acts')
            doctype = "Act"
        except NoSuchElementException:
                pass 

    # Get citations of the case 
    no_of_citation, cited_content = get_citations(driver)

    # Clean the content and write into a text vile
    if content_element != "":
        content = content_element.text
        content = content.replace(junk_txt, '')

        file_name = f"document_{doc_id}.txt"
        file_name = os.path.join(txtdownload_path, file_name)
        with open(file_name, 'w+', encoding='utf-8') as file:
            file.write("Meta Data\nDocType: "+ doctype + "\nURL: "+url + '\n')
            file.write("Cites: " + str(cited_content) + '\n+++++++++++++++++++++++++\n\n')
            file.write(content + '\n')

            # pdf file automatically saved with some auto created name, that is not in our control. pdf_name is that pdf file name.
            pdf_name = content.split('\n')[2]
            pdf_name = re.sub(r'[&\/()\[\]{},\'\"\s.]', '_', pdf_name)
            pdf_name = re.sub(r'\_+', '_', pdf_name)
            pdf_name = pdf_name + ".PDF"

            #file.write(href)
        
        #prepare metedata of the document for a csv file
        data.append({"id":doc_id, "file_name": file_name,'pdf_name': pdf_name, "doctype": doctype, "word_count": len(content.replace('\n',' ').split(' ')), "url":url, "citations":no_of_citation, "cites":str(cited_content)})
        status = f"Content saved to {file_name}"
    else:
        pdf_name = ""
        data.append({"id":doc_id, "file_name": "", "pdf_name":"", "doctype": "", "word_count":0, "url":"", "citations":"", "cites":""})
        status = f"Document {url} does not exist"
    return status, pdf_name   

In [7]:
#debugging 
# pdf_name = """Bhulan & 3 Ors. vs "Ram" Raj's (Judicial ... on 8 January, 2010"""
# pdf_name = re.sub(r'[&\/()\[\]{},\'\"\s.]', '_', pdf_name)
# pdf_name = re.sub(r'\_+', '_', pdf_name)
# pdf_name = pdf_name + ".PDF"
# pdf_name 

In [8]:
def download_pdf(doc_id, driver,pdf_name):
    # Navigate to the webpage
    url = f'https://indiankanoon.org/doc/{doc_id}/'
    driver.get(url)

    # Wait for the page to load
    time.sleep(1)

    # Find the button element by its XPath
    try:
        button = driver.find_element(By.XPATH, "//input[@value='Get this document in PDF']")

        # Click on the button
        button.click()

        # Wait for the download to start
        time.sleep(1)
        status = f"PDF file saved: {pdf_name}"
        
    except:
        status = f"PDF Document Not Found : {url}"
        pdf_name = ""
        pass
    return status, pdf_name

In [13]:
# Main Body.

# set website root here
base_url = 'https://indiankanoon.org/doc/'

# Set the pdf download path for chrome
download_path    = r'D:\github\1-Projects\Research\SSBM\IndianKannon\PDF_IndianKanoon'

# Set the txt download path
txtdownload_path = r'D:\github\1-Projects\Research\SSBM\IndianKannon\TXT_IndianKanoon'

start = 3501
num_webpages = 1500

data=[]
driver = getDriver()


i=0

for doc_id in range(start, start+num_webpages):
# for doc_id in random_numbers:
    url = base_url + str(doc_id) + '/'
    #print(url)

    driver.get(url)

    status1, pdf_name = savein_textfile(doc_id, url, driver, data)

    status2, pdf_name = download_pdf(doc_id, driver, pdf_name)
    print (f"{status1}, {status2}")
    
    if i // 50 == i / 50:
        time.sleep(3)
        # df=pd.DataFrame(data)
        # df.to_csv(f'IndiaKannon{i}.csv')
    i+=1

# Close the browser and the service
driver.quit()

end = start+num_webpages-1
pd.DataFrame(data).to_csv(f"IndianKannon-{start}-{end}.csv")

Content saved to D:\github\1-Projects\Research\SSBM\IndianKannon\TXT_IndianKanoon\document_3501.txt, PDF file saved: Sh_Parminder_Singh_Son_Of_Sh_vs_Sh_G_S_Bhatia_Proprietor_Of_on_4_December_2007.PDF
Content saved to D:\github\1-Projects\Research\SSBM\IndianKannon\TXT_IndianKanoon\document_3502.txt, PDF file saved: M_S_Achint_Enterprises_vs_Railway_Board_Ministry_Of_on_25_August_2010.PDF
Content saved to D:\github\1-Projects\Research\SSBM\IndianKannon\TXT_IndianKanoon\document_3503.txt, PDF file saved: Pradeep_Kumar_Rajgarhia_vs_The_Commissioner_Of_Income_Tax_on_3_February_2011.PDF
Content saved to D:\github\1-Projects\Research\SSBM\IndianKannon\TXT_IndianKanoon\document_3504.txt, PDF file saved: Well_Knit_Apparels_P_Ltd_vs_Commissioner_Of_Customs_on_23_October_1998.PDF
Content saved to D:\github\1-Projects\Research\SSBM\IndianKannon\TXT_IndianKanoon\document_3505.txt, PDF file saved: Narendra_Nath_Roy_And_Ors_vs_Abani_Kumar_Roy_And_Ors_on_1_June_1937.PDF
Content saved to D:\github\1-P

In [10]:
pd.DataFrame(data)

,id,file_name,pdf_name,doctype,word_count,url,citations,cites
0,2501,D:\github\1-Projects\Research\SSBM\IndianKanno...,Kishorbhai_vs_Chandulal_on_25_November_2010.PDF,Judgement,396,https://indiankanoon.org/doc/2501/,,[]
1,2502,D:\github\1-Projects\Research\SSBM\IndianKanno...,Birendra_Das_vs_The_State_Of_Bihar_on_27_Septe...,Judgement,295,https://indiankanoon.org/doc/2502/,,[]
2,2503,D:\github\1-Projects\Research\SSBM\IndianKanno...,Assistant_Commissioner_vs_Nandanam_Constructio...,Judgement,1762,https://indiankanoon.org/doc/2503/,Cites 4 docs,['Deputy Commissioner Of Sales Tax ... vs Thom...
3,2504,D:\github\1-Projects\Research\SSBM\IndianKanno...,The_Commissioner_Commercial_vs_M_S_Sanjay_Trad...,Judgement,68,https://indiankanoon.org/doc/2504/,,[]
4,2505,D:\github\1-Projects\Research\SSBM\IndianKanno...,Sukh_Pal_vs_State_Of_Rajasthan_And_Ors_on_3_Ja...,Judgement,2864,https://indiankanoon.org/doc/2505/,Cites 8 docs,['Section 302 in The Indian Penal Code [Comple...
...,...,...,...,...,...,...,...,...
995,3496,D:\github\1-Projects\Research\SSBM\IndianKanno...,Krishna_Kumar_Yadav_@_Jhinku_Yadav_vs_Sri_Subh...,Judgement,97,https://indiankanoon.org/doc/3496/,,[]
996,3497,D:\github\1-Projects\Research\SSBM\IndianKanno...,Commissioner_Of_Income-Tax_vs_Birla_Bros_P_Ltd...,Judgement,2101,https://indiankanoon.org/doc/3497/,Cites 3 docs,"['The Income- Tax Act, 1995 : https://indianka..."
997,3498,D:\github\1-Projects\Research\SSBM\IndianKanno...,Shiv_Govind_vs_The_State_Of_Madhya_Pradesh_on_...,Judgement,2445,https://indiankanoon.org/doc/3498/,Cites 10 docs,['The Indian Penal Code\nCentral Government Ac...
998,3499,D:\github\1-Projects\Research\SSBM\IndianKanno...,Dr_Sandeep_Mehta_vs_Ministry_Of_Railways_on_20...,Judgement,846,https://indiankanoon.org/doc/3499/,,[]


# Ignore the code below.

In [11]:
# working with keyboards

# # Simulate key presses to save the file
# pyautogui.hotkey('alt', 's')
# time.sleep(1)
# pyautogui.typewrite(download_path)
# time.sleep(1)
# pyautogui.press('enter')

# Wait for the download to complete
# time.sleep(1)

In [12]:
# new_file_name = '1960213.pdf'
# downloaded_file_path = os.path.join(download_path, 'doc.pdf')
# os.rename(downloaded_file_path, os.path.join(download_path, new_file_name))